<a href="https://colab.research.google.com/github/ColeJMitchell/machine-learning-stock-prediction/blob/main/lstm_model_and_random_simulated_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from termcolor import colored
from IPython.display import clear_output
!pip install tensorflow
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import random
clear_output()

# **Loading The Data**

In [9]:
TESTDATADIR = "test_stocks/"
TESTSTOCKS = [
    "AAPL", # Apple
    "AMD",  # AMD
    "AMZN", # Amazon
    "META", # Meta Platforms
    "NFLX", # Netflix
    "QCOM", # Qualcomm
    "SBUX", # Starbucks
    "SCSO", # Cisco
    "TSLA", # Tesla
]
predictions = {}
for stock in TESTSTOCKS:
    predictions[stock] = pd.read_parquet(TESTDATADIR + stock + "_results.parquet")

# **Paper Trading Method 2 : Using The LSTM Model**

In [58]:
#Load the LSTM model and simulate it by paper trading
model = load_model("stock_prediction.h5", compile = False)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,513 (119.19 KB)

 Trainable params: 30,321 (118.44 KB)

 Non-trainable params: 192 (768.00 B)

In [11]:
# Data Format
print(predictions['AAPL'])

          Date  Actual_Scaled  Prediction_Scaled  Actual  Prediction
0   2025-03-25       0.592725           0.532328  223.75  218.519653
1   2025-03-24       0.557852           0.553596  220.73  220.361404
2   2025-03-21       0.529446           0.521709  218.27  217.599976
3   2025-03-20       0.481293           0.500552  214.10  215.767776
4   2025-03-19       0.494457           0.464448  215.24  212.641190
..         ...            ...                ...     ...         ...
215 2024-05-14       0.173326           0.243720  187.43  193.526138
216 2024-05-13       0.160046           0.241477  186.28  193.331909
217 2024-05-10       0.122748           0.240314  183.05  193.231171
218 2024-05-09       0.140300           0.237339  184.57  192.973541
219 2024-05-08       0.119169           0.238530  182.74  193.076721

[220 rows x 5 columns]


## Preprocessing The Data With Sliding Windows To Match Shape Required By The LSTM

In [12]:
prediction_windows = {}
actual_values = {}
window_size = 30

# Stocks need to remain separate for the paper trading
for stock in TESTSTOCKS:
    prediction_windows[stock] = []
    actual_values[stock] = []
    for i in range(0, 220 - window_size + 1, window_size):
        window = predictions[stock]["Prediction"][i:i + window_size]
        prediction_windows[stock].append(np.array(window))
        actual_values[stock].append(predictions[stock]["Actual"][i+window_size])

print(f"Shape of Stock Sliding Windows: {np.array(prediction_windows[TESTSTOCKS[0]]).shape}")
print(f"Shape of Corresponding Actual Value Array: {np.array(actual_values[TESTSTOCKS[0]]).shape}")

Shape of Stock Sliding Windows: (7, 30)
Shape of Corresponding Actual Value Array: (7,)


## Trading Algorithm Using The LSTM's Predictions

In [13]:
# A list of stocks purchased by the LSTM model is saved so it can know when to sell, the model starts with the same amount of money as the random model - 10,000 dollars
portfolio = {}
initial_wallet = 10000

for stock in TESTSTOCKS:
  portfolio[stock] = {"Holding": 1000, "Bought": 0, "Purchase_Prices": [], "Final_Amount_Sold":0}
# If a stock is predicted to rise in the future (larger value then final entry in sliding windo), it is bought by the model, half of the allocated money for the stock is bought (1000 dollars)
# If the stock is predicted to fall below the purchase price the n
for stock in TESTSTOCKS:
  windows = prediction_windows[stock]
  for window in windows:
    baseline_price = window[-1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_window = scaler.fit_transform(window.reshape(-1,1))
    prediction = model.predict(normalized_window.reshape(1,30), verbose=0)
    prediction = scaler.inverse_transform(prediction)
    current_holdings = portfolio[stock]["Holding"]
    current_bought = portfolio[stock]["Bought"]
    if baseline_price < prediction.item():
      if current_holdings - 100 <= 0:
        continue
      else:
        # Stocks are bought in 100 dollar increments
        portfolio[stock]["Holding"] =  portfolio[stock]["Holding"] - 100
        portfolio[stock]["Bought"] = portfolio[stock]["Bought"] + 100
        portfolio[stock]["Purchase_Prices"].append(baseline_price)
    elif baseline_price > prediction.item():
      if current_bought <= 0:
        continue
      else:
        for i in range (len(portfolio[stock]["Purchase_Prices"])):
          # Sells the bought stock at the new price if it is worse than one of the previously purchase price points
          if portfolio[stock]["Purchase_Prices"][i] > prediction.item():
            reduced_percentage = prediction.item() / portfolio[stock]["Purchase_Prices"][i]
            portfolio[stock]["Holding"] =  portfolio[stock]["Holding"] + (100 * reduced_percentage)
            portfolio[stock]["Bought"] = portfolio[stock]["Bought"] - 100
            del portfolio[stock]["Purchase_Prices"][i]
            break
    else:
      pass

  #Sells all the stocks that were bought at the very end
  final_stock_price = float(windows[-1][-1])
  for purchase_price in portfolio[stock]["Purchase_Prices"]:
    purchase_price = float(purchase_price)
    shares_bought = 100 / purchase_price
    sale_proceeds = shares_bought * final_stock_price
    portfolio[stock]["Final_Amount_Sold"] += sale_proceeds

## Profit And Loss Margins For The Model

In [14]:
# Output the final earnings and losses
for stock in TESTSTOCKS:
  print(f"{stock} detailed stats:")
  print(f"Final Holdings: {portfolio[stock]['Holding']}")
  print(f"Stock Quantity Bought: {portfolio[stock]['Bought']}")
  print(f"Stock Purchase Prices: {[float(x) for x in portfolio[stock]['Purchase_Prices']]}")
  print(f"Final Amount Sold: {[float(portfolio[stock]['Final_Amount_Sold'])]}")
  print("\n")

AAPL detailed stats:
Final Holdings: 800
Stock Quantity Bought: 200
Stock Purchase Prices: [220, 193]
Final Amount Sold: 187


AMD detailed stats:
Final Holdings: 900
Stock Quantity Bought: 100
Stock Purchase Prices: [151]
Final Amount Sold: 100


AMZN detailed stats:
Final Holdings: 700
Stock Quantity Bought: 300
Stock Purchase Prices: [186, 180, 182]
Final Amount Sold: 298


META detailed stats:
Final Holdings: 789
Stock Quantity Bought: 200
Stock Purchase Prices: [531, 514]
Final Amount Sold: 196


NFLX detailed stats:
Final Holdings: 770
Stock Quantity Bought: 200
Stock Purchase Prices: [734, 721]
Final Amount Sold: 198


QCOM detailed stats:
Final Holdings: 800
Stock Quantity Bought: 200
Stock Purchase Prices: [157, 193]
Final Amount Sold: 223


SBUX detailed stats:
Final Holdings: 690
Stock Quantity Bought: 300
Stock Purchase Prices: [94, 92, 81]
Final Amount Sold: 278


SCSO detailed stats:
Final Holdings: 893
Stock Quantity Bought: 100
Stock Purchase Prices: [49]
Final Amount S

In [15]:
# Final amount made and lost
for stock in TESTSTOCKS:
  print(f"{stock} overall stats:")
  print(f"Starting Amount: 1000")
  print(f"Ending Amount: {float(portfolio[stock]['Final_Amount_Sold']) + portfolio[stock]['Holding']}")
  print("\n")

AAPL overall stats:
Starting Amount: 1000
Ending Amount: 987.6563684570206


AMD overall stats:
Starting Amount: 1000
Ending Amount: 1000.0


AMZN overall stats:
Starting Amount: 1000
Ending Amount: 998.6328655166965


META overall stats:
Starting Amount: 1000
Ending Amount: 986.09814453125


NFLX overall stats:
Starting Amount: 1000
Ending Amount: 969.0552368164062


QCOM overall stats:
Starting Amount: 1000
Ending Amount: 1023.0339017256974


SBUX overall stats:
Starting Amount: 1000
Ending Amount: 969.3707275390625


SCSO overall stats:
Starting Amount: 1000
Ending Amount: 994.380126953125


TSLA overall stats:
Starting Amount: 1000
Ending Amount: 968.0418701171875


-----------------------------------------------------------------------------------------------
Total Beginning: 10,000 USD

Total End: 8172.58984375 USD

Percent Change From Initial Wallet: 0.8172590136528015
